<a href="https://colab.research.google.com/github/Nathan-Roll1/ProsodPy/blob/main/Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount necessary data files from Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone __init__.py functions for preprocessing
!git clone https://github.com/Nathan-Roll1/ProsodPy

from ProsodPy.ProsodPy import *
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import librosa   
import pickle

# plotting options intended for 20x3 dimensions
plt.rcParams["figure.figsize"] = (20,3)

In [7]:
# path to resampled discourse audio
init_path = 'drive/Shareddrives/Ling Thesis/resampled/'

# get name of each file
fs = os.listdir(init_path)

# merge names with path to generate full path
fs = [init_path + x for x in fs]

In [ ]:
# path to transcripts
text_path = 'drive/Shareddrives/Rezonator Data/SBCSAE/'

# for every audio file...
for n, f in enumerate(fs):
  print(n,f)

  # get matching transcript file
  tt = f.split('/')[-1].split('_')[0]
  txt = text_path + tt +'.cha'

  # identify boundary timestamps
  boundaries = get_boundaries(txt)

  # adjust for 8k sample rate
  boundaries = boundaries*8000 

  # load audio file
  with open(f, 'rb') as handle:
    audio_array = pickle.load(handle)
    
  # generate features and labels
  pre_mfcc, post_mfcc, boundary_mfcc, labels = MFCC_preprocess(audio_array, boundaries)

  # negative sample false labels to equalize classes
  size = sum(labels==1)
  indicies = list(np.random.choice(np.where(labels == 0)[0], int(size)))
  indicies += list(np.where(labels == 1)[0])

  # apply selected indices to featuresets and labels
  labels = labels[indicies] 
  pre_mfcc = pre_mfcc[indicies]
  post_mfcc = post_mfcc[indicies]
  boundary_mfcc = boundary_mfcc[indicies]

  # save features
  for t in ['pre_mfcc','post_mfcc','boundary_mfcc']:
    with open(f'drive/Shareddrives/Ling Thesis/PREPROCESSED/{t}/{tt}_cont.pickle', 'wb') as handle:
      pickle.dump(globals()[t], handle, protocol=pickle.HIGHEST_PROTOCOL)

  # save labels
  with open(f'drive/Shareddrives/Ling Thesis/PREPROCESSED/labels/{tt}_cont.pickle', 'wb') as handle:
    pickle.dump(labels, handle, protocol=pickle.HIGHEST_PROTOCOL)